# FITCHECK PROCESSING

In [ ]:
import os
# # Loop through the folder to find .xlsx files
base_folder = "Base_folder
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith(".xlsx"):
            file_path = os.path.join(root, file)
            print(f'file_path = "{file_path}"')

In [ ]:
# Defining the Variables here
import os

# file_path = "path_to_collector.xlsx"


In [ ]:
# Create an empty text file named "vm_stats.txt" in the same folder as the Excel file
import os
print(file_path)

# Create the text file in the same directory as the Excel file
txt_file_path = os.path.join(os.path.dirname(file_path), "vm_stats.txt")

with open(txt_file_path, "w") as f:
    f.write("hello")
    print("file created!!!")
    pass  # Just create the file, do not write anything

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Get a list of all sheet names in the Excel file
sheet_names = pd.ExcelFile(file_path).sheet_names

# Print all sheet names
print("Available sheets in the Excel file:")
for i, sheet in enumerate(sheet_names, 1):
    print(f"{i}. {sheet}")

# Count the number of sheets
print(f"\nTotal number of sheets: {len(sheet_names)}")

## MEMORY

In [ ]:
sheet_name = "vMemory"  # Replace with your actual sheet name
df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')

new_df = df[['VM Name', 'Size (MiB)', 'Peak %', 'Average %']]
df = new_df.sort_values(by='Peak %', ascending=False)

# Set up bar positions
bar_width = 0.25
x = np.arange(len(df["VM Name"]))

# Plot
plt.figure(figsize=(16, 8))
plt.bar(x - bar_width, df["Peak %"], width=bar_width, label="Peak %", color="#7855fa")
plt.bar(x, df["Average %"], width=bar_width, label="Average %", color="#131313")
plt.bar(x + bar_width, df["Size (MiB)"]/1000, width=bar_width, label="Size (GB)", color="#aaaaaa")

# Formatting
plt.xticks(x, df["VM Name"], rotation=90)
plt.ylabel("Memory Usage (%)")
plt.title("Memory Usage and Allocation per VM")
plt.legend()
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Add text annotations for memory size in GB
for i, v in enumerate(df['Size (MiB)']):
    size_gb = v/1024  # Convert MiB to GiB
    plt.text(i + bar_width, v/1000 + 0.5, f"{size_gb:.1f} GB", 
             color='black', fontweight='bold', ha='center', va='bottom', 
             rotation=90, fontsize=8)

# Save the plot as an image in the same directory as the Excel file
plot_path = os.path.join(os.path.dirname(file_path), "memory_usage.png")
plt.savefig(plot_path)
plt.show()

# Create a new dataframe for memory analysis
memory_analysis = df.copy()

# Calculate memory utilization thresholds
memory_analysis['Over 80% Avg'] = memory_analysis['Average %'] > 80
memory_analysis['Under 20% Avg'] = memory_analysis['Average %'] < 20

# Identify high memory usage VMs
high_memory_vms = memory_analysis[memory_analysis['Over 80% Avg'] == True]
high_mem_lines = [f"VMs with AVERAGE memory usage exceeding 80%:"]
if len(high_memory_vms) > 0:
    for index, row in high_memory_vms.iterrows():
        high_mem_lines.append(f"{row['VM Name']}: {row['Average %']:.2f}% (of {row['Size (MiB)']/1024:.1f} GB)")
else:
    high_mem_lines.append("No VMs found with average memory usage exceeding 80%")

# Identify low memory usage VMs
low_memory_vms = memory_analysis[memory_analysis['Under 20% Avg'] == True]
low_mem_lines = [f"\nVMs with AVERAGE memory usage under 20%:"]
if len(low_memory_vms) > 0:
    low_mem_lines.append(f"Total count: {len(low_memory_vms)} VMs")
    # Sort by memory size to highlight large underutilized VMs first
    sorted_low_mem = low_memory_vms.sort_values(by='Size (MiB)', ascending=False)
    # List all underutilized VMs
    for index, row in sorted_low_mem.iterrows():
        low_mem_lines.append(f"{row['VM Name']}: {row['Average %']:.2f}% (of {row['Size (MiB)']/1024:.1f} GB)")
else:
    low_mem_lines.append("No VMs found with average memory usage under 20%")

# Write stats to vm_stats.txt in the same directory as the Excel file
stats_file_path = os.path.join(os.path.dirname(file_path), "vm_stats.txt")
with open(stats_file_path, "w") as f:
    for line in high_mem_lines + low_mem_lines:
        f.write(line + "\n")

# Print to console as before
for line in high_mem_lines:
    print(line)
for line in low_mem_lines:
    print(line)

## CPU DATA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sheet_name = "vCPU"  # Replace with your actual sheet name
df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
new_df = df[['VM Name', 'vCPUs', 'Peak %', 'Average %']]
# new_df.head(20)

# Sort the data by Peak % to better visualize VM performance
sorted_df = new_df.sort_values(by='Peak %', ascending=False)

# Set up bar positions
bar_width = 0.25
x = np.arange(len(sorted_df))

# Create plot with multiple metrics
plt.figure(figsize=(16, 10))

# Plot bars for each metric
plt.bar(x - bar_width, sorted_df['Peak %'], width=bar_width, label='Peak %', color='#7855fa')
plt.bar(x, sorted_df['Average %'], width=bar_width, label='Average %', color='#131313')
plt.bar(x + bar_width, sorted_df['vCPUs']/2, width=bar_width, label='vCPUs (scaled ÷2)', color='#aaaaaa')

# Add formatting
plt.xlabel('VM Name')
plt.ylabel('Usage %')
plt.title('CPU Usage and vCPU Allocation by VM')
plt.xticks(x, sorted_df['VM Name'], rotation=90)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Add text annotations for vCPU counts
for i, v in enumerate(sorted_df['vCPUs']):
    plt.text(i + bar_width, v/2 + 0.5, str(v), color='black', fontweight='bold', 
             ha='center', va='bottom', rotation=0, fontsize=8)

# Save the CPU plot as an image in the same directory as the Excel file
cpu_plot_path = os.path.join(os.path.dirname(file_path), "cpu_usage.png")
plt.savefig(cpu_plot_path)
plt.show()


# Create a dataframe for CPU analysis
cpu_analysis = sorted_df.copy()

# Calculate CPU utilization thresholds
cpu_analysis['Over 80% Peak'] = cpu_analysis['Peak %'] > 80
cpu_analysis['Over 80% Avg'] = cpu_analysis['Average %'] > 80
cpu_analysis['Under 20% Avg'] = cpu_analysis['Average %'] < 20

# Identify high CPU usage VMs (peak)
high_cpu_peak = cpu_analysis[cpu_analysis['Over 80% Peak'] == True]
print(f"VMs with PEAK CPU usage exceeding 80%:")
if len(high_cpu_peak) > 0:
    print(f"Total count: {len(high_cpu_peak)} VMs")
    for index, row in high_cpu_peak.head(10).iterrows():
        print(f"{row['VM Name']}: {row['Peak %']:.2f}% peak, {row['Average %']:.2f}% avg ({row['vCPUs']} vCPUs)")
    if len(high_cpu_peak) > 10:
        print(f"... and {len(high_cpu_peak) - 10} more VMs with >80% peak CPU usage")
else:
    print("No VMs found with peak CPU usage exceeding 80%")

# Identify high CPU usage VMs (average)
high_cpu_avg = cpu_analysis[cpu_analysis['Over 80% Avg'] == True]
print(f"\nVMs with AVERAGE CPU usage exceeding 80%:")
if len(high_cpu_avg) > 0:
    for index, row in high_cpu_avg.iterrows():
        print(f"{row['VM Name']}: {row['Average %']:.2f}% avg, {row['Peak %']:.2f}% peak ({row['vCPUs']} vCPUs)")
else:
    print("No VMs found with average CPU usage exceeding 80%")

# Identify low CPU usage VMs
low_cpu_vms = cpu_analysis[cpu_analysis['Under 20% Avg'] == True]
print(f"\nVMs with AVERAGE CPU usage under 20%:")
if len(low_cpu_vms) > 0:
    print(f"Total count: {len(low_cpu_vms)} VMs")
    # Show top 10 VMs with high vCPU count but low usage
    sorted_low_cpu = low_cpu_vms.sort_values(by='vCPUs', ascending=False)
    for index, row in sorted_low_cpu.head(10).iterrows():
        print(f"{row['VM Name']}: {row['Average %']:.2f}% avg, {row['Peak %']:.2f}% peak ({row['vCPUs']} vCPUs)")
    if len(low_cpu_vms) > 10:
        print(f"... and {len(low_cpu_vms) - 10} more VMs with <20% average CPU usage")
else:
    print("No VMs found with average CPU usage under 20%")

# Calculate potential CPU consolidation
total_vcpus = sorted_df['vCPUs'].sum()
low_usage_vcpus = low_cpu_vms['vCPUs'].sum()
print(f"\nPotential CPU optimization:")
print(f"Total vCPUs across all VMs: {total_vcpus}")
print(f"vCPUs in low-usage VMs (<20% avg): {low_usage_vcpus} ({low_usage_vcpus/total_vcpus*100:.2f}% of total)")

# Prepare CPU stats for writing
cpu_stats_lines = []
# High CPU usage (peak)
cpu_stats_lines.append("\nVMs with PEAK CPU usage exceeding 80%:")
if len(high_cpu_peak) > 0:
    cpu_stats_lines.append(f"Total count: {len(high_cpu_peak)} VMs")
    for index, row in high_cpu_peak.iterrows():
        cpu_stats_lines.append(f"{row['VM Name']}: {row['Peak %']:.2f}% peak, {row['Average %']:.2f}% avg ({row['vCPUs']} vCPUs)")
else:
    cpu_stats_lines.append("No VMs found with peak CPU usage exceeding 80%")
# High CPU usage (average)
cpu_stats_lines.append("\nVMs with AVERAGE CPU usage exceeding 80%:")
if len(high_cpu_avg) > 0:
    for index, row in high_cpu_avg.iterrows():
        cpu_stats_lines.append(f"{row['VM Name']}: {row['Average %']:.2f}% avg, {row['Peak %']:.2f}% peak ({row['vCPUs']} vCPUs)")
else:
    cpu_stats_lines.append("No VMs found with average CPU usage exceeding 80%")
# Low CPU usage
cpu_stats_lines.append("\nVMs with AVERAGE CPU usage under 20%:")
if len(low_cpu_vms) > 0:
    cpu_stats_lines.append(f"Total count: {len(low_cpu_vms)} VMs")
    sorted_low_cpu = low_cpu_vms.sort_values(by='vCPUs', ascending=False)
    for index, row in sorted_low_cpu.iterrows():
        cpu_stats_lines.append(f"{row['VM Name']}: {row['Average %']:.2f}% avg, {row['Peak %']:.2f}% peak ({row['vCPUs']} vCPUs)")
else:
    cpu_stats_lines.append("No VMs found with average CPU usage under 20%")
# CPU optimization
cpu_stats_lines.append("\nPotential CPU optimization:")
cpu_stats_lines.append(f"Total vCPUs across all VMs: {total_vcpus}")
cpu_stats_lines.append(f"vCPUs in low-usage VMs (<20% avg): {low_usage_vcpus} ({low_usage_vcpus/total_vcpus*100:.2f}% of total)")

# Append CPU stats to vm_stats.txt
stats_file_path = os.path.join(os.path.dirname(file_path), "vm_stats.txt")
with open(stats_file_path, "a") as f:
    for line in cpu_stats_lines:
        f.write(line + "\n")

## DISK DATA

In [ ]:
# Read the vDisk sheet from Excel file
sheet_name = "vmList"  # Use the vDisk sheet instead of vCPU
df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')

# Extract relevant columns for disk analysis
new_df = df[['VM Name', 'Capacity (MiB)', 'Consumed (MiB)']]

# Calculate percentage of disk used and remaining space
new_df['Consumed %'] = (new_df['Consumed (MiB)'] / new_df['Capacity (MiB)']) * 100
new_df['Remaining (MiB)'] = new_df['Capacity (MiB)'] - new_df['Consumed (MiB)']

# Create color mapping based on usage percentage
colors = []
for pct in new_df['Consumed %']:
    if pct > 90:
        colors.append('red')
    elif pct > 80:
        colors.append('orange')
    elif pct > 70:
        colors.append('yellow')
    else:
        colors.append('#7855fa')  # Use the same blue color as in previous plots

# Convert MiB to GB for plotting
new_df['Capacity (GB)'] = new_df['Capacity (MiB)'] / 1024
new_df['Consumed (GB)'] = new_df['Consumed (MiB)'] / 1024
new_df['Remaining (GB)'] = new_df['Remaining (MiB)'] / 1024

# Sort the dataframe by Consumed % to easily identify high-usage VMs
sorted_by_usage = new_df.sort_values(by='Consumed %', ascending=False)

# Filter VMs with usage > 80%
high_usage_vms = sorted_by_usage[sorted_by_usage['Consumed %'] > 80]

# Print the high usage VMs
print("VMs with storage usage exceeding 80%:")
if len(high_usage_vms) > 0:
    for index, row in high_usage_vms.iterrows():
        print(f"{row['VM Name']}: {row['Consumed %']:.2f}% used ({row['Consumed (GB)']:.2f} GB / {row['Capacity (GB)']:.2f} GB)")
else:
    print("No VMs found with storage usage exceeding 80%")

# Plot storage usage in GB
plt.figure(figsize=(16, 7))
x = np.arange(len(new_df))
bar_width = 0.4

# Use colors list for bar colors
plt.bar(x, new_df['Consumed (GB)'], width=bar_width, label='Consumed (GB)', color=colors)
plt.bar(x, new_df['Capacity (GB)'], width=bar_width, label='Capacity (GB)', color='#131313', alpha=0.2)

plt.xlabel('VM Name')
plt.ylabel('Storage (GB)')
plt.title('VM Storage Usage (GB)')
plt.xticks(x, new_df['VM Name'], rotation=90, fontsize=8)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()

# Add text annotations vertically
for i, (consumed, remaining, percentage) in enumerate(zip(new_df['Consumed (GB)'], 
                                                         new_df['Remaining (GB)'],
                                                         new_df['Consumed %'])):
    plt.text(i, consumed + 2, f"{remaining:.1f} GB\n({percentage:.1f}%)", 
            ha='center', va='bottom', fontsize=7, color='black', rotation=90)

# Save the disk plot as an image in the same directory as the Excel file
# (Save BEFORE plt.show() to avoid blank image)
disk_plot_path = os.path.join(os.path.dirname(file_path), "disk_usage.png")
plt.savefig(disk_plot_path)
plt.show()

# Prepare disk stats for writing
disk_stats_lines = []
disk_stats_lines.append("\nVMs with storage usage exceeding 80%:")
if len(high_usage_vms) > 0:
    disk_stats_lines.append(f"Total count: {len(high_usage_vms)} VMs")
    for index, row in high_usage_vms.iterrows():
        disk_stats_lines.append(f"{row['VM Name']}: {row['Consumed %']:.2f}% used ({row['Consumed (GB)']:.2f} GB / {row['Capacity (GB)']:.2f} GB)")
else:
    disk_stats_lines.append("No VMs found with storage usage exceeding 80%")

# Append disk stats to vm_stats.txt
stats_file_path = os.path.join(os.path.dirname(file_path), "vm_stats.txt")
with open(stats_file_path, "a") as f:
    for line in disk_stats_lines:
        f.write(line + "\n")